<a href="https://colab.research.google.com/github/leehgkor/lhg/blob/main/(5)_%ED%86%A0%ED%94%BD%EB%AA%A8%EB%8D%B8%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Step 2: 필요한 라이브러리 설치 및 임포트

!pip install konlpy pandas scikit-learn pyLDAvis gensim

import os
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models.ldamodel import LdaModel
from gensim.matutils import Sparse2Corpus
from gensim.corpora.dictionary import Dictionary
from collections import defaultdict
import pyLDAvis.gensim_models
import numpy as np


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.9/493.9 kB 26.4 MB/s eta 0:00:00


In [ ]:
# Step 3: 텍스트 데이터 읽기 및 전처리(태그셋이 없는 말뭉치라 그냥 진행)

# 텍스트 파일 읽기
corpus = []

folder_path = "/content/drive/MyDrive/txt_u8_half"  # txt 파일 폴더 경로

for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):
        with open(os.path.join(folder_path, filename), "r", encoding="utf-8") as file:
            lines = file.readlines()
            corpus.append(" ".join(line.strip() for line in lines).strip())  # 모든 줄을 본문으로 처리

print(f"총 문서 수: {len(corpus)}")

# 본문 샘플 출력
print("샘플 본문:", corpus[:3])  # 상위 3개 본문 출력


/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


총 문서 수: 34
샘플 본문: ['치숙(痴叔)  채만식   우리 아저씨 말이지요, 아따 저 거시키, 한참 당년에 무엇이냐 그놈의 것, 사회주의라더냐, 막걸리라더냐 그걸 하다, 징역 살고 나와서 폐병으로 시방 앓고 누웠는 우리 오촌 고모부 그 양반……  머, 말두 마시오. 대체 사람이 어쩌면 글쎄…… 내 원!  신세 간 데 없지요.  자, 십 년 적공, 대학교까지 공부한 것 풀어먹지도 못했지요, 좋은 청춘 어영부영 다 보냈지요, 신분에는 전과자라는 붉은 도장 찍혔지요, 몸에는 몹쓸 병까지 들었지요, 이 신세를 해가지굴랑은 굴속 같은 오두막집 단간 셋방 구석에서 사시장철 밤이나 낮이나 눈 따악감고 드러누웠군요.  재산이 어디 집 터전인들 있을 턱이 있나요. 서발 막대 내저어야 짚검불 하나 걸리는 것 없는 철빈(鐵貧)인데.  우리 아주머니가, 그래도 그 아주머니가, 어질고 얌전해서 그 알뜰한 남편양반 받드느라 삯바느질이야, 남의 집 품빨래야, 화장품 장사야, 그 칙살스런 벌이를 해다가 겨우겨우 목구멍에 풀칠을 하지요.  어디루 대나 그 양반은 죽는 게 두루 좋은 일인데 죽지도 아니해요.  우리 아주머니가 불쌍해요. 아, 진작 한 나이라도 젊어서 팔자를 고치는 게 아니라, 무슨 놈의 수난 후분을 바라고 있다가 고생을 하는지.  근 이십 년 소박을 당했지요.  이십 년을 섧은 청춘 한숨으로 보내고서 다아 늦게야 송장 여대치게 생긴 그 양반을 그래도 남편이라고 모셔다가는 병 수종들으랴, 먹고 살랴, 애가 진하고 다니는 걸 보면 참말 가엾어요.  그게 무슨 죄다짐이람? 팔자, 팔자 하지만 왜 팔자를 고치지를 못하고서 그래요. 죄선(朝鮮) 구식 부인네들은 다아 문명을 못하고 깨지를 못해서 그러지.  그 양반이 한시바삐 죽기나 했으면 우리 아주머니는 차라리 신세 편하리다.  심덕 좋겠다, 솜씨 얌전하겠다 하니 어디 가선들 제가 일신 몸 가누고 편안히 못 지내요?  가만있자, 열 여섯 살에 아저씨네 집으로 시집을 갔다니깐 그게 내가 세살 적이니 꼬박 열 여덟 해로군. 열 여덟

In [ ]:
# Step 4: 형태소 분석을 통한 토크나이징

# 형태소 분석기 초기화(Okt 사용)
tokenizer = Okt()

def tokenize_and_clean(text):
    tokens = tokenizer.nouns(text)  # 명사만 추출
    return tokens

# 토큰화된 텍스트
tokenized_corpus = [" ".join(tokenize_and_clean(text)) for text in corpus]


/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Step 5: TF-IDF를 활용한 임베딩

custom_stopwords = [
  "그리고", "하지만", "그런데", "삼이", "보고", "모양", "아내", "사내", "거기", "거리", "그냥", "자리", "조금", "여기", "저기", "동안", "오늘", "다음", "한번", "이번", "무엇", "이제", "처음", "여자", "소리", "가지", "그때", "자신", "머리", "아이", "수가", "얼굴", "또한", "이런", "저런", "그것", "사람", "생각", "그이", "자기", "우리", "어디", "때문", "모두", "무슨", "누구", "다른", "하나", "다시", "지금", "시작", "듯이", "그녀", "이야기"
]

# TF-IDF 벡터화
tfidf_vectorizer = TfidfVectorizer(max_features=30, stop_words=custom_stopwords)  # 최대 단어 수 제한
X_tfidf = tfidf_vectorizer.fit_transform(tokenized_corpus)
tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()

# 확인용 출력
print(f"TF-IDF 임베딩 완료! 총 단어 수: {len(tfidf_feature_names)}")


TF-IDF 임베딩 완료! 총 단어 수: 30


/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Step 6: LDA 모델 실행

# TF-IDF 데이터를 Gensim 형식으로 변환
corpus_gensim = Sparse2Corpus(X_tfidf, documents_columns=False)  # Gensim Corpus
id2word = Dictionary([tfidf_feature_names.tolist()])  # 단어 ID-매핑

# Gensim LDA 모델 생성
num_topics = 5 # 토픽 수 설정
gensim_lda = LdaModel(corpus=corpus_gensim, num_topics=num_topics, id2word=id2word, random_state=42)

print("\n주요토픽 확인:")
for idx, topic in gensim_lda.print_topics(num_words=5):
    words = [word.split("*")[1].strip().strip('"') for word in topic.split(" + ")]
    print(f"토픽 {idx + 1}: {', '.join(words)}")

print("\n문서별 토픽 분포 확인:")
for doc_idx, (topic_dist, original_text) in enumerate(zip(gensim_lda[corpus_gensim], corpus)):
    # 확률 값을 백분율로 변환하고 % 기호 추가
    topic_dist_percent = [(topic_id, f"{round(prob * 100, 2)}%") for topic_id, prob in topic_dist]
    print(f"문서 {doc_idx + 1:02d} ({original_text[:15]}...): {topic_dist_percent}")



/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



주요토픽 확인:
토픽 1: 서울, 화공, 조선, 가슴, 세상
토픽 2: 소녀, 어머니, 아들, 젊은이, 혼자
토픽 3: 영감, 조선, 성기, 관모, 서울
토픽 4: 아저씨, 동네, 아버지, 조선, 서울
토픽 5: 마음, 노인, 세상, 조선, 영감

문서별 토픽 분포 확인:
문서 01 (치숙(痴叔)  채만식   우...): [(0, '8.67%'), (1, '8.67%'), (2, '8.81%'), (3, '64.9%'), (4, '8.96%')]
문서 02 (붉은산  김동인   그것은 ...): [(0, '7.46%'), (1, '7.55%'), (2, '7.51%'), (3, '69.83%'), (4, '7.65%')]
문서 03 (채만식   <미스터 방>  ...): [(0, '5.09%'), (1, '5.11%'), (2, '21.24%'), (3, '63.49%'), (4, '5.07%')]
문서 04 (복덕방(福德房)   이태준 ...): [(0, '5.63%'), (1, '5.81%'), (2, '76.72%'), (3, '5.94%'), (4, '5.89%')]
문서 05 (등신불  김동리  등신불(等...): [(0, '4.86%'), (1, '80.37%'), (2, '4.86%'), (3, '4.93%'), (4, '4.98%')]
문서 06 (바위  김동리   북쪽 하늘...): [(0, '6.25%'), (1, '73.98%'), (2, '6.66%'), (3, '6.46%'), (4, '6.64%')]
문서 07 (해방전후(解放前後)   이태...): [(0, '5.67%'), (1, '5.89%'), (2, '76.73%'), (3, '5.81%'), (4, '5.9%')]
문서 08 (술 권하는 사회(社會)  현...): [(0, '5.95%'), (1, '5.75%'), (2, '5.95%'), (3, '5.92%'), (4, '76.43%')]
문서 09 (논 이야기   채만식   일...): [(0, '6.05%'), (1, 

In [ ]:
#Step 7: LDA 시각화
#Intertopic Distance Map은 LDA 모델에서 추출된 토픽 간의 관계(유사도 또는 거리)를 다차원 축소 기법(MDS)을 사용하여 2차원 평면에 시각화한 것.
#PyLDAVis 라이브러리를 사용하여 생성되며, 각 토픽은 원으로 표현되고, 원의 크기는 토픽의 비중을 나타냄.

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(gensim_lda, corpus_gensim, id2word)
pyLDAvis.display(vis)


/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# 추가: 토픽별 문서 분류 결과 확인

document_topic_mapping = defaultdict(list)

print("\n문서별 주요 토픽 확인 및 분류:")
for doc_idx, (topic_dist, original_text) in enumerate(zip(gensim_lda[corpus_gensim], corpus)):
    # 가장 높은 비율의 토픽을 찾기
    dominant_topic = max(topic_dist, key=lambda x: x[1])  # (토픽ID, 확률) 중 확률이 가장 높은 것
    topic_id, topic_percent = dominant_topic
    topic_percent_str = f"{round(topic_percent * 100, 2)}%"  # 백분율로 변환

    # 결과 출력
    print(f"문서 {doc_idx + 1:02d} ({original_text[:5]}...): 주요 토픽 {topic_id} ({topic_percent_str})")

    # 주요 토픽 기준으로 문서를 분류
    document_topic_mapping[topic_id].append(f"문서 {doc_idx + 1:02d} ({original_text[:15]}...)")

# 토픽별로 문서 분류 결과 출력
print("\n토픽별 문서 분류 결과:")
for topic, documents in document_topic_mapping.items():
    print(f"토픽 {topic}:")
    for doc in documents:
        print(f"  - {doc}")



문서별 주요 토픽 확인 및 분류:
문서 01 (치숙(痴叔...): 주요 토픽 2 (84.36%)
문서 02 (붉은산  ...): 주요 토픽 1 (87.58%)
문서 03 (채만식  ...): 주요 토픽 2 (85.11%)
문서 04 (복덕방(福...): 주요 토픽 3 (84.99%)
문서 05 (등신불  ...): 주요 토픽 2 (84.83%)
문서 06 (바위  김...): 주요 토픽 2 (85.33%)
문서 07 (해방전후(...): 주요 토픽 0 (89.96%)
문서 08 (술 권하는...): 주요 토픽 4 (85.32%)
문서 09 (논 이야기...): 주요 토픽 1 (84.12%)
문서 10 (줄 -「줄...): 주요 토픽 0 (87.75%)
문서 11 (할머니의 ...): 주요 토픽 2 (81.32%)
문서 12 (B사감과 ...): 주요 토픽 4 (86.97%)
문서 13 (배따라기 ...): 주요 토픽 3 (82.35%)
문서 14 (태형(笞刑...): 주요 토픽 3 (89.31%)
문서 15 (광화사 김...): 주요 토픽 4 (81.02%)
문서 16 (감자  김...): 주요 토픽 2 (76.68%)
문서 17 (레디메이드...): 주요 토픽 0 (91.61%)
문서 18 (잔인한 도...): 주요 토픽 3 (86.6%)
문서 19 (화랑의 후...): 주요 토픽 3 (86.17%)
문서 20 (운수 좋은...): 주요 토픽 1 (85.3%)
문서 21 (역마(驛馬...): 주요 토픽 3 (80.98%)
문서 22 (눈길   ...): 주요 토픽 2 (85.78%)
문서 23 (패강냉(浿...): 주요 토픽 4 (84.88%)
문서 24 (병신과 머...): 주요 토픽 2 (86.54%)
문서 25 (까마귀  ...): 주요 토픽 1 (87.76%)
문서 26 (고향  현...): 주요 토픽 2 (89.83%)
문서 27 (돌다리  ...): 주요 토픽 0 (88.56%)
문서 28 (영월 영감...): 주요 토픽 4 (84.33%)
문서

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
